In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])



gs


### CommaSeparatedListOutputParser


In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
print(prompt)

input_variables=['question'] input_types={} partial_variables={'format_instructions': 'Return a JSON object.'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, template='#Format: {format_instructions}\n\n#Question: {question}'), additional_kwargs={})]


In [3]:
# OpenAI 모델 설정
#model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))

[
    {
        "mission_name": "뉴호라이즌스",
        "goal": "명왕성 탐사",
        "agency": "NASA"
    },
    {
        "mission_name": "카시니-호이겐스",
        "goal": "토성의 위성 타이탄 탐사",
        "agency": "NASA, ESA, 이탈리아 우주국"
    },
    {
        "mission_name": "창어 4호",
        "goal": "달의 뒷면 탐사",
        "agency": "중국 우주국"
    }
]


### JsonOutputParser


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학 분야 전문가 AI입니다. 질문에 대해 체계적이고 간결한 답변을 JSON 형식으로 제공하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# JSON 출력 형식 지침을 프롬프트에 적용
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [8]:
# OpenAI 모델 설정
# model = ChatOpenAI(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)
# 프롬프트, 모델, 출력 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 질문 설정 (우주 탐사 관련 질문)
question = "최근 10년간 진행된 주요 우주 탐사 미션 3가지를 알려주세요. \
각 미션의 이름은 `mission_name`에, 목표는 `goal`에, 주관 기관은 `agency`에 담아 주세요."

# 체인 실행 및 JSON 응답 받기
response = chain.invoke({"question": question})

# JSON 데이터 출력
print(json.dumps(response, indent=4, ensure_ascii=False))

[
    {
        "mission_name": "뉴호라이즌스",
        "goal": "명왕성 탐사",
        "agency": "NASA"
    },
    {
        "mission_name": "카시니-호이겐스",
        "goal": "토성의 위성 타이탄 탐사",
        "agency": "NASA, ESA, 이탈리아 우주국"
    },
    {
        "mission_name": "창어 4호",
        "goal": "달의 뒷면 탐사",
        "agency": "중국 우주국"
    }
]


### PandasDataFrameOutputParser


In [9]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import re

# Titanic 데이터셋 로드
df = pd.read_csv('data/titanic.csv')

# Pandas DataFrame Output Parser 설정
parser = PandasDataFrameOutputParser(dataframe=df)

# 형식 지침 출력
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

Format Instructions:
 The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2,

In [ ]:
# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model_name="gpt-4o")
# model = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0
# )

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template=""" 
    You are a helpful assistant that interacts with a Pandas DataFrame.
    The DataFrame contains the following columns: {columns}.
    
    Your task is to answer the user's query by generating a command in the following format:
    {format_instructions}
    
    User Query: {query}    
    """,
    input_variables=["query"],
    partial_variables={
        "format_instructions": format_instructions,
        "columns": ", ".join(df.columns)
    },
)
print(prompt.partial_variables['columns'])

Survived, Pclass, Name, Sex, Age, Siblings/Spouses Aboard, Parents/Children Aboard, Fare


In [11]:
# 체인 생성
chain = prompt | model | parser

# 모델 응답 받기
try:
    # **Name 열을 표시하십시오.**
    print('Name 컬럼 출력')
    df_query = "Show the Name column"

    # parser_output = chain.invoke({"query": df_query})
    # print(type(parser_output))
    # print(parser_output)

        # **첫번째 행을 표시하십시오.**
    print('첫번째 행 출력')
    df_query2 = "Show first row"

    parser_output2 = chain.invoke({"query": df_query2})
    print(parser_output2)

except Exception as e:
    print(f"오류 발생: {e}")

Name 컬럼 출력
첫번째 행 출력
오류 발생: Request 'The operation "head" is typically used to show the first few rows, but in this case, you want to show only the first row. A well-formatted command for this would be:

head:0

However, since you specifically asked to "Show first row", I will use the "iloc" operation which is used for integer-location based indexing.

The command will be:
 
row:0 

If you want to get a specific column for the first row, you can do:

row:0[Survived] 

Let me know if you need anything else!' is not correctly formatted.                     Please refer to the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [5]:
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# OpenAI 모델 초기화
model = ChatOpenAI(temperature=0, model_name="gpt-4o")
# model = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0
# )

# 응답 스키마 정의
response_schemas = [
    ResponseSchema(name="data", description="A list of dictionaries representing table rows."),
]

# Output Parser 설정
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    You are an AI assistant that generates tabular data. 
    You must return the data in JSON format that follows this schema:
    
    {format_instructions}
        
    **User Query:**
    {query}
    """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [6]:

# 체인 생성 (프롬프트 → 모델 → OutputParser)
chain = prompt | model | parser

# 실행 함수
def generate_dataframe(user_query):
    try:
        # 모델 호출
        json_response = chain.invoke({"query": user_query})

        # 모델이 반환한 JSON을 Pandas DataFrame으로 변환
        df = pd.DataFrame(json_response["data"])

        # 결과 출력
        print("\n🔹 Generated DataFrame:\n")
        return df

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None

In [ ]:
# [예제 1] 2024년 상반기 서울 아파트 평균 매매 가격 데이터 생성
print('2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성')
df_seoul_housing = generate_dataframe(
    "Create a dataset of the average apartment sale prices in Seoul for the second half of 2024 with columns: District (구), Average Price (in KRW), Number of Transactions, and Year-over-Year Change (%)."
)
print(df_seoul_housing.shape)
df_seoul_housing

2024년 하반기 서울 아파트 평균 매매 가격 데이터 생성

🔹 Generated DataFrame:

(8, 4)


,District,Average Price,Number of Transactions,Year-over-Year Change (%)
3,Seocho-gu,1450000000,1100,3.8
4,Songpa-gu,1300000000,1050,3.2
5,Yongsan-gu,1250000000,900,2.9
6,Gwanak-gu,800000000,700,1.5
7,Dongdaemun-gu,850000000,750,2.0


In [2]:
print('2024년 서울 지하철역별 유동 인구 데이터')
# [예제 2] 2024년 서울 지하철역별 유동 인구 데이터
df_seoul_subway = generate_dataframe(
    "Generate a dataset of the top 10 busiest subway stations in Seoul in 2024 with columns: Station Name, Line Number, Daily Passenger Volume, and Weekday vs Weekend Ratio."
)
if df_seoul_subway is not None:
    df_seoul_subway.head()

2024년 서울 지하철역별 유동 인구 데이터


NameError: name 'generate_dataframe' is not defined

In [13]:
print('한국 5대 편의점 브랜드별 2024년 매출 및 점포 수')
# [예제 3] 한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
df_korean_convenience_stores = generate_dataframe(
    "Create a dataset of the top 5 convenience store brands in Korea in 2024 with columns: Brand Name, Number of Stores, Total Revenue (in billion KRW), and Market Share (%)."
)
df_korean_convenience_stores.head()

한국 5대 편의점 브랜드별 2024년 매출 및 점포 수
❌ 오류 발생: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}


AttributeError: 'NoneType' object has no attribute 'head'

### PydanticOutputParser

In [7]:
# poetry add pydantic
# %pip install pydantic 

from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

In [8]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    movie_title: str = Field(description="추천 영화 제목")
    reason: str = Field(description="추천 이유")
    genre: List[str] = Field(description="영화 장르")
    estimated_rating: float = Field(description="10점 만점에서 예상 평점")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 영화를 추천해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [9]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
query = "1990년대 클래식한 느낌의 공포 영화 추천해줘"
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"추천 영화: {output.movie_title}")
print(f"추천 이유: {output.reason}")
print(f"장르: {', '.join(output.genre)}")
print(f"예상 평점: {output.estimated_rating}/10")

추천 영화: The Sixth Sense (1999)
추천 이유: 1990년대 대표적인 심리 공포 영화로, 반전의 묘미가 있는 클래식한 느낌의 영화입니다.
장르: 공포, 미스터리, 스릴러
예상 평점: 8.5/10


### StructuredOutputParser

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="pros", description="리뷰에서 언급된 장점 3가지를 리스트로 출력"),
    ResponseSchema(name="cons", description="리뷰에서 언급된 단점 3가지를 리스트로 출력"),
    ResponseSchema(name="summary", description="리뷰를 한 문장으로 요약")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 테스트 리뷰 데이터
review = """
이 스마트폰은 배터리 수명이 정말 좋아서 하루 종일 사용해도 충전이 필요 없었어요. 
카메라 화질도 선명하고, 특히 야간 모드가 훌륭합니다. 
다만 가격이 조금 비싸고, 무게가 200g이 넘어서 손이 피곤할 수 있어요.
"""

# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rating": string  // 5점 만점에서 예상 평점
	"pros": string  // 리뷰에서 언급된 장점 3가지를 리스트로 출력
	"cons": string  // 리뷰에서 언급된 단점 3가지를 리스트로 출력
	"summary": string  // 리뷰를 한 문장으로 요약
}
```
===== 분석 결과 =====
{'cons': ['가격이 비싸다',
          '무게가 무거워서 손이 피곤할 수 있다',
          '단점 3번째 항목이 없습니다. 대신 기재할 내용이 없습니다.'],
 'pros': ['배터리 수명이 좋다', '카메라 화질이 선명하다', '야간 모드가 훌륭하다'],
 'rating': '4',
 'summary': '스마트폰의 배터리 수명과 카메라 화질에 만족했지만, 가격과 무게가 다소 부담스럽다는 리뷰입니다.'}


### DatetimeOutputParser


In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime, timedelta

print(datetime.today())

# 출력 파서 초기화 (시간대 포함 가능)
datetime_parser = DatetimeOutputParser()
format_instructions = datetime_parser.get_format_instructions()

print("날짜 출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
# 현재 날짜를 명시적으로 프롬프트에 주입
current_date = datetime.now().strftime("%Y-%m-%d")
template = f"""
현재 날짜: {current_date}
다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.
텍스트: {{text}}

{{format_instructions}}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)


2025-06-11 14:35:26.834442
날짜 출력 형식 지시사항:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1534-11-20T22:09:52.514526Z, 0666-08-12T22:46:39.780258Z, 0033-01-16T09:50:34.682341Z

Return ONLY this string, no other words!


In [12]:

# 모델 초기화 (temperature=0.1로 설정해 정확한 날짜 출력 강조)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.1
)

# 테스트 데이터 (다양한 날짜 형식 포함)
texts = [
    "회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.",
    "프로젝트 마감일은 다음 주 금요일입니다.",
    "행사 시작: 7/25/2025 18:00 KST",
    "3일 후에 시스템 점검이 진행됩니다."
]

# 체인 실행 및 결과 출력
chain = prompt | model | datetime_parser

for text in texts:
    print(f"\n원본 텍스트: {text}")
    output = chain.invoke({"text": text})
    print(f"추출된 날짜: {output.strftime('%Y-%m-%d %H:%M:%S %Z')}")

    
# 이벤트 추출용 프롬프트
current_date = datetime.now().strftime("%Y-%m-%d")

event_template = """
현재 날짜: {current_date}
다음 텍스트에서 모든 이벤트의 날짜/시간을 추출하세요. 각 이벤트는 이름과 날짜를 포함해야 합니다.
텍스트: {text}

출력 형식:
- 이벤트명: [이름]
- 날짜: [YYYY-MM-DD HH:MM:SS]
"""

event_prompt = ChatPromptTemplate.from_template(event_template)
event_chain = event_prompt | model

# 예시 텍스트 (여러 이벤트 포함)
event_text = """
12월 10일에 크리스마스 마켓이 열리고, 12월 24일에는 크리스마스 이브 파티가 있습니다.
또한 1월 1일 00:00에 새해 카운트다운이 진행될 예정입니다.
"""

print(event_chain.invoke({"current_date":current_date, "text": event_text}).content)


원본 텍스트: 회의는 2025년 6월 15일 오후 2시에 예정되어 있습니다.
추출된 날짜: 2025-06-15 14:00:00 

원본 텍스트: 프로젝트 마감일은 다음 주 금요일입니다.
추출된 날짜: 2025-06-20 00:00:00 

원본 텍스트: 행사 시작: 7/25/2025 18:00 KST
추출된 날짜: 2025-07-25 18:00:00 

원본 텍스트: 3일 후에 시스템 점검이 진행됩니다.
추출된 날짜: 2025-06-14 00:00:00 
이 텍스트에서 이벤트 날짜/시간 추출:

- 이벤트명: 크리스마스 마켓
- 날짜: 2025-12-10 00:00:00

- 이벤트명: 크리스마스 이브 파티
- 날짜: 2025-12-24 00:00:00

- 이벤트명: 새해 카운트다운
- 날짜: 2025-01-01 00:00:00
